# Hyperparameter Tuning with the HParams

### We will use HParams dashboard in TensorBoard to tune the hyperparameters for the model

In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

#### Load the dataset

In [2]:
import json
from flair.data import Sentence
import progressbar
import pickle
import sys

path_dir = "/mnt/d/ASE/HLTProject/BioASQ2020"

if path_dir not in sys.path:
    sys.path.append(path_dir)

In [3]:
from data import load_data_yesno, generate_embeddings_yesno, generate_embeddings_yesno_pooling, load_embeddings

In [4]:
data=load_data_yesno("../data/training8b.json")

In [5]:
from flair.embeddings import ELMoEmbeddings

embeddings_elmo_pubmed = ELMoEmbeddings('pubmed') 

In [24]:
from flair.embeddings import DocumentPoolEmbeddings, Sentence
from tensorflow.keras.layers import Dense

pooling_model = DocumentPoolEmbeddings([embeddings_elmo_pubmed])

In [66]:
import time
import datetime

In [26]:
# start = time.time()
# embeddings = generate_embeddings_yesno_pooling(pooling_model, data, "embedding_yes_no.emb")
# end = time.time()
embeddings = load_embeddings("embedding_yes_no.emb")

In [27]:
print(end - start)

NameError: name 'end' is not defined

In [28]:
import numpy as np
VALIDATION_SPLIT = 0.33

In [29]:
emb_numpy = np.array(embeddings)
indices = np.arange(emb_numpy.shape[0])
np.random.shuffle(indices)

In [30]:
# Data includes questions (0) and snippets (2)
data = emb_numpy[indices,0::2]
labels = np.array(emb_numpy[indices,1], dtype=np.float)
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

data = np.array([np.concatenate([el[0].data, el[1].data]) for el in data])

x_develop = data[:-nb_validation_samples]
y_develop = labels[:-nb_validation_samples]

x_train = x_develop[:-nb_validation_samples]
y_train = y_develop[:-nb_validation_samples]

# Hold out validation
x_val = x_develop[-nb_validation_samples:]
y_val = y_develop[-nb_validation_samples:]

# Hold out test
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

### List the hyperparameters to be tuned and their possible values 

In [67]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 60, 70, 80]))
HP_NUM_HIDDENS = hp.HParam('num_hiddens', hp.Discrete([1,2,3,4,5]))
HP_ACT_FUN = hp.HParam('act_fun', hp.Discrete(['sigmoid', 'tanh', 'relu']))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
# HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.2]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))


METRIC_ACCURACY = 'accuracy'

directory_results = 'logs/hparam_tuning'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

with tf.summary.create_file_writer(directory_results).as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_NUM_HIDDENS, HP_ACT_FUN, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [68]:
hparams = {
    HP_NUM_UNITS: HP_NUM_UNITS.domain.values,
    HP_NUM_HIDDENS: HP_NUM_HIDDENS.domain.values,
    HP_ACT_FUN: HP_ACT_FUN.domain.values,
    # HP_DROPOUT: HP_DROPOUT.domain.values,
    HP_OPTIMIZER: HP_OPTIMIZER.domain.values,
}

BATCH_SIZE = 32
EPOCHS = 10

### We define a function that wraps the model definition (Two dense layers with a dropout in between them) and its training.

In [105]:
def train_test_model(hparams, logdir):
  # model = tf.keras.models.Sequential([
  #   tf.keras.layers.Flatten(),
  #   tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
  #   tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
  #   tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  # ])
  model = tf.keras.models.Sequential()
  for i in range(hparams[HP_NUM_HIDDENS]):
    model.add(Dense(hparams[HP_NUM_UNITS], activation = hparams[HP_ACT_FUN]))

  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, 
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = (x_val, y_val),
    callbacks=[
            tf.keras.callbacks.TensorBoard(logdir),  # log metrics
            hp.KerasCallback(logdir, hparams),  # log hparams
        ]) # Run with 1 epoch to speed things up for demo purposes

  _, accuracy = model.evaluate(x_val, y_val)
  
  return accuracy

In [106]:
result = train_test_model(params_list[0], 'logs/hparam_tuning20200614-192023')

In [88]:
result

[0.6931496262550354, 0.7862069010734558]

### For each run, log an hparams summary with the hyperparameters and final accuracy:

### Now we can run the experiments. Results are reported in the log directory

In [70]:
from itertools import product

In [74]:
def grid_search(grid_params, inputs = None, targets = None):
    '''
        grid_search function.
        It performs a grid search using the given parameters.
        It reports result on a file

        Parameters
        ----------
        - grid_params: A dictionary containing the parameters to test
        - inputs: A list containing the neural network inputs 
        - targets: A list containing the neural network targets 

        Returns
        -------
        - min_error: The minimum error obtained
        - best_params: The best hyperparameters
    '''
    
    keys, values = zip(*grid_params.items())
    params_list = [dict(zip(keys, v)) for v in product(*values)]
    print(len(params_list))
    
    for session_num, hparams in enumerate(params_list):
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)

        print({h.name: hparams[h] for h in hparams})
        
        # run_dir = 'logs/hparam_tuning/' + run_name
        run_dir = directory_results + "/"+ run_name
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, run_dir)


In [75]:
start = time.time()
grid_search(hparams)
end = time.time()

=========] - 0s 27ms/step - loss: 3.1579 - accuracy: 0.8040 - val_loss: 3.3954 - val_accuracy: 0.7897
Epoch 7/10
10/10 [==============================] - 0s 19ms/step - loss: 3.1573 - accuracy: 0.8040 - val_loss: 3.3955 - val_accuracy: 0.7897
Epoch 8/10
10/10 [==============================] - 0s 29ms/step - loss: 3.1571 - accuracy: 0.8040 - val_loss: 3.3954 - val_accuracy: 0.7897
Epoch 9/10
10/10 [==============================] - 0s 17ms/step - loss: 3.1570 - accuracy: 0.8040 - val_loss: 3.3955 - val_accuracy: 0.7897
Epoch 10/10
10/10 [==============================] - 0s 3ms/step - loss: 3.3950 - accuracy: 0.7897
--- Starting trial: run-108
{'num_units': 80, 'num_hiddens': 4, 'act_fun': 'relu', 'optimizer': 'adam'}
Epoch 1/10
10/10 [==============================] - 0s 30ms/step - loss: 3.3938 - accuracy: 0.5415 - val_loss: 1.8121 - val_accuracy: 0.7655
Epoch 2/10
10/10 [==============================] - 0s 32ms/step - loss: 1.3444 - accuracy: 0.8040 - val_loss: 0.8219 - val_accurac

In [76]:
keys, values = zip(*hparams.items())
params_list = [dict(zip(keys, v)) for v in product(*values)]
print(len(params_list))
print(end - start)

120
265.37332463264465


### Visualize results in tensorboard

In [77]:
# The command is the following:
# tensorboard --logdir _test/logs/hparam_tuning

In [78]:
directory_results

'logs/hparam_tuning20200614-192023'

In [79]:
# !tensorboard --logdir logs/hparam_tuning20200614-192023

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.2.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C
